<a href="https://colab.research.google.com/github/jeremy10230004/ThurthdayAiClass/blob/main/%E4%BA%BA%E5%B7%A5%E6%96%87%E6%9C%AC1202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
! pip install pyLDAvis=="2.1.2"
! pip install nltk

     |████████████████████████████████| 1.6 MB 5.3 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97738 sha256=80a6bceefdefde15bc2882f97ef26a5c150628a99bd96168d668597a8d502a84
  Stored in directory: /root/.cache/pip/wheels/3b/fb/41/e32e5312da9f440d34c4eff0d2207b46dc9332a7b931ef1e89
Successfully built pyLDAvis


In [3]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#1introduction
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)


In [5]:
stopwords = stopwords.words("english")
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [6]:
data = load_data("gdrive/My Drive/Colab Notebooks/ushmm_dn.json")["texts"]

print(data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [7]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [8]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [9]:
#BIGRAMS AND TRIGRAMS
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=100)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)

print (data_bigrams_trigrams[0][0:20])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [10]:
#TF-IDF REMOVAL
from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]
# print (corpus[0][0:20])

tfidf = TfidfModel(corpus, id2word=id2word)

low_value = 0.03
words  = []
words_missing_in_tfidf = []
for i in range(0, len(corpus)):
    bow = corpus[i]
    low_value_words = [] #reinitialize to be safe. You can skip this.
    tfidf_ids = [id for id, value in tfidf[bow]]
    bow_ids = [id for id, value in bow]
    low_value_words = [id for id, value in tfidf[bow] if value < low_value]
    drops = low_value_words+words_missing_in_tfidf
    for item in drops:
        words.append(id2word[item])
    words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids] # The words with tf-idf socre 0 will be missing

    new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
    corpus[i] = new_bow

In [72]:
"""
zid2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)
"""

'\nzid2word = corpora.Dictionary(data_words)\n\ncorpus = []\nfor text in data_words:\n    new = id2word.doc2bow(text)\n    corpus.append(new)\n\nprint (corpus[0][0:20])\n\nword = id2word[[0][:1][0]]\nprint (word)\n'

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,id2word=id2word,num_topics=30,random_state=100,update_every=1,chunksize=100,passes=10,alpha="auto")

In [12]:
#lda_model.save("models/test_model.model")
new_model = lda_model

In [78]:
#new_model = gensim.models.ldamodel.LdaModel.load("models/test_model.model")

In [13]:
test_doc = corpus[-1]

vector = new_model[test_doc]
print (vector)

def Sort(sub_li):
    sub_li.sort(key = lambda x: x[1])
    sub_li.reverse()
    return (sub_li)
new_vector = Sort(vector)
print (new_vector)

[(0, 0.0898911), (1, 0.13099748), (5, 0.018294744), (6, 0.0132175), (13, 0.3886215), (15, 0.011563489), (16, 0.15825169), (21, 0.012481257), (27, 0.043595523), (28, 0.13016799)]
[(13, 0.3886215), (16, 0.15825169), (1, 0.13099748), (28, 0.13016799), (0, 0.0898911), (27, 0.043595523), (5, 0.018294744), (6, 0.0132175), (21, 0.012481257), (15, 0.011563489)]


In [15]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
0      0.358723  0.140401       1        1  8.860338
16     0.111912  0.225026       2        1  7.880903
15    -0.241811 -0.060481       3        1  6.300121
19    -0.406446  0.017275       4        1  6.138916
21    -0.096247 -0.324688       5        1  6.113928
10    -0.209089 -0.366233       6        1  6.100675
12    -0.360254 -0.206410       7        1  5.988628
23    -0.269624 -0.263463       8        1  5.871300
4      0.244586  0.338168       9        1  4.346052
14    -0.406015 -0.113363      10        1  4.345832
9      0.384111 -0.199895      11        1  3.704018
7     -0.210462  0.123463      12        1  2.872990
3      0.358535  0.230970      13        1  2.658498
24    -0.087888  0.361488      14        1  2.529297
5      0.111949  0.369852      15        1  2.433628
8      0.008495  0.420410      16        1  2.417836
13    -0.211498  0.366604      17        1  2.357174
17    -0.309825  0.241398      18        1  2.196992
18     0.400036  0.021757      19        1  2.109752
1      0.092757 -0.343315      20        1  1.963799
25     0.368254 -0.077168      21        1  1.691801
11    -0.075998  0.161843      22        1  1.685443
26    -0.068467 -0.099310      23        1  1.599065
29    -0.343879  0.126885      24        1  1.582001
2      0.043439 -0.179467      25        1  1.500364
27     0.230022 -0.177841      26        1  1.422729
20     0.082132 -0.441438      27        1  1.331591
6      0.174220  0.017482      28        1  0.856576
22     0.069460  0.035654      29        1  0.759138
28     0.258873 -0.345601      30        1  0.380616, topic_info=             Term         Freq        Total Category  logprob  loglift
867           use  1562.000000  1562.000000  Default  30.0000  30.0000
677      remember   966.000000   966.000000  Default  29.0000  29.0000
506         maybe   785.000000   785.000000  Default  28.0000  28.0000
2299       ghetto  2181.000000  2181.000000  Default  27.0000  27.0000
583          over   717.000000   717.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
743        sister     0.011682   598.751525  Topic30 -11.0514  -5.2734
8387     dominant     0.011682     6.664950  Topic30 -11.0514  -0.7755
356   grandfather     0.011680   167.090064  Topic30 -11.0516  -3.9973
112       brother     0.011680   816.142968  Topic30 -11.0516  -5.5833
533        mother     0.011680   544.724550  Topic30 -11.0516  -5.1791

[2084 rows x 6 columns], token_table=       Topic      Freq                  Term
term                                        
7406      12  0.925037            abbreviate
0          1  0.002325                  able
0          2  0.006975                  able
0          4  0.174385                  able
0          5  0.476651                  able
...      ...       ...                   ...
2557      19  0.533736               zionist
3496       5  0.750639  zionist_organization
3496       8  0.214468  zionist_organization
14964     22  0.698598                zlotys
8759      22  0.675868                  zoom

[4681 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 17, 16, 20, 22, 11, 13, 24, 5, 15, 10, 8, 4, 25, 6, 9, 14, 18, 19, 2, 26, 12, 27, 30, 3, 28, 21, 7, 23, 29])